In [59]:
#pip install scrapbook

In [60]:
#pip install recommenders

In [61]:
#pip install recommenders[examples]

In [62]:
# set the environment path to find Recommenders
from tempfile import TemporaryDirectory
import sys
import os
import pandas as pd
import numpy as np
import scrapbook as sb
import torch, fastai
from fastai.collab import collab_learner, CollabDataBunch, load_learner

from recommenders.utils.constants import (
    DEFAULT_USER_COL as USER, 
    DEFAULT_ITEM_COL as ITEM, 
    DEFAULT_RATING_COL as RATING, 
    DEFAULT_TIMESTAMP_COL as TIMESTAMP, 
    DEFAULT_PREDICTION_COL as PREDICTION
) 
from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.fastai.fastai_utils import cartesian_product, score
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.evaluation.python_evaluation import rmse, mae, rsquared, exp_var

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Fast AI version: {}".format(fastai.__version__))
print("Torch version: {}".format(torch.__version__))
print("Cuda Available: {}".format(torch.cuda.is_available()))
print("CuDNN Enabled: {}".format(torch.backends.cudnn.enabled))

System version: 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
Pandas version: 1.3.5
Fast AI version: 1.0.61
Torch version: 1.10.0+cu111
Cuda Available: False
CuDNN Enabled: True


In [63]:
# top k items to recommend
TOP_K = 5

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'

# Model parameters
N_FACTORS = 10
EPOCHS = 5

In [64]:
ratings_df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=[USER,ITEM,RATING,TIMESTAMP]
)

# make sure the IDs are loaded as strings to better prevent confusion with embedding ids
ratings_df[USER] = ratings_df[USER].astype('str')
ratings_df[ITEM] = ratings_df[ITEM].astype('str')

ratings_df.head()

,userID,itemID,rating,timestamp
0,1,1193,5.0,978300760
1,1,661,3.0,978302109
2,1,914,3.0,978301968
3,1,3408,4.0,978300275
4,1,2355,5.0,978824291


100%|██████████| 5.78k/5.78k [00:00<00:00, 17.6kKB/s]


,userID,itemID,rating,timestamp
0,1,1193,5.0,978300760
1,1,661,3.0,978302109
2,1,914,3.0,978301968
3,1,3408,4.0,978300275
4,1,2355,5.0,978824291


In [93]:
ratings_df

,userID,itemID,rating,timestamp
0,1,1193,5.0,978300760
1,1,661,3.0,978302109
2,1,914,3.0,978301968
3,1,3408,4.0,978300275
4,1,2355,5.0,978824291
...,...,...,...,...
1000204,6040,1091,1.0,956716541
1000205,6040,1094,5.0,956704887
1000206,6040,562,5.0,956704746
1000207,6040,1096,4.0,956715648


In [65]:
# Split the dataset
train_valid_df, test_df = python_stratified_split(
    ratings_df, 
    ratio=0.8, 
    min_rating=1, 
    filter_by="item", 
    col_user=USER, 
    col_item=ITEM
)

In [66]:
# Remove "cold" users from test set  
test_df = test_df[test_df.userID.isin(train_valid_df.userID)]

# Training

In [67]:
# fix random seeds to make sure our runs are reproducible
np.random.seed(101)
torch.manual_seed(101)
torch.cuda.manual_seed_all(101)

In [68]:
with Timer() as preprocess_time:
    data = CollabDataBunch.from_df(train_valid_df, 
                                   user_name=USER, 
                                   item_name=ITEM, 
                                   rating_name=RATING, 
                                   valid_pct=0)

In [94]:
data.show_batch()

userID,itemID,target
4577,1307,4.0
1685,260,4.0
2485,3723,4.0
4572,258,4.0
3778,2700,3.0


In [70]:
learn = collab_learner(data, n_factors=N_FACTORS, y_range=[0,5.5], wd=1e-1)
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(6041, 10)
  (i_weight): Embedding(3707, 10)
  (u_bias): Embedding(6041, 1)
  (i_bias): Embedding(3707, 1)
)

EmbeddingDotBias(
  (u_weight): Embedding(6041, 10)
  (i_weight): Embedding(3707, 10)
  (u_bias): Embedding(6041, 1)
  (i_bias): Embedding(3707, 1)
)

In [71]:
with Timer() as train_time:
    learn.fit_one_cycle(EPOCHS, max_lr=5e-3)

print("Took {} seconds for training.".format(train_time))

epoch,train_loss,valid_loss,time
0,0.918599,#na#,01:42
1,0.916762,#na#,01:46
2,0.884625,#na#,01:47
3,0.823717,#na#,01:46
4,0.838749,#na#,01:44


Took 527.6200 seconds for training.


In [72]:
tmp = TemporaryDirectory()
model_path = os.path.join(tmp.name, "movielens_model.pkl")

In [73]:
learn.export(model_path)

#Generating Recommendations

In [74]:
learner = load_learner(tmp.name, "movielens_model.pkl")

In [75]:
total_users, total_items = learner.data.train_ds.x.classes.values()
total_items = total_items[1:]
total_users = total_users[1:]

In [76]:
test_users = test_df[USER].unique()
test_users = np.intersect1d(test_users, total_users)

In [77]:
users_items = cartesian_product(np.array(test_users),np.array(total_items))
users_items = pd.DataFrame(users_items, columns=[USER,ITEM])

In [78]:
training_removed = pd.merge(users_items, train_valid_df.astype(str), on=[USER, ITEM], how='left')
training_removed = training_removed[training_removed[RATING].isna()][[USER, ITEM]]

Score the model to find the top K recommendation


In [79]:
with Timer() as test_time:
    top_k_scores = score(learner, 
                         test_df=training_removed,
                         user_col=USER, 
                         item_col=ITEM, 
                         prediction_col=PREDICTION)

print("Took {} seconds for {} predictions.".format(test_time, len(training_removed)))

Took 33.0534 seconds for 21550935 predictions.


In [80]:
eval_map = map_at_k(test_df, top_k_scores, col_user=USER, col_item=ITEM, 
                    col_rating=RATING, col_prediction=PREDICTION, 
                    relevancy_method="top_k", k=TOP_K)

In [81]:
eval_ndcg = ndcg_at_k(test_df, top_k_scores, col_user=USER, col_item=ITEM, 
                      col_rating=RATING, col_prediction=PREDICTION, 
                      relevancy_method="top_k", k=TOP_K)

In [82]:
eval_precision = precision_at_k(test_df, top_k_scores, col_user=USER, col_item=ITEM, 
                                col_rating=RATING, col_prediction=PREDICTION, 
                                relevancy_method="top_k", k=TOP_K)

In [83]:
eval_recall = recall_at_k(test_df, top_k_scores, col_user=USER, col_item=ITEM, 
                          col_rating=RATING, col_prediction=PREDICTION, 
                          relevancy_method="top_k", k=TOP_K)

In [84]:
print("Model:\t" + learn.__class__.__name__,
      "Top K:\t%d" % TOP_K,
      "MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Model:	CollabLearner
Top K:	5
MAP:	0.015392
NDCG:	0.143051
Precision@K:	0.136528
Recall@K:	0.027491


In [85]:
scores = score(learner, 
               test_df=test_df.copy(), 
               user_col=USER, 
               item_col=ITEM, 
               prediction_col=PREDICTION)

In [86]:
eval_r2 = rsquared(test_df, scores, col_user=USER, col_item=ITEM, col_rating=RATING, col_prediction=PREDICTION)
eval_rmse = rmse(test_df, scores, col_user=USER, col_item=ITEM, col_rating=RATING, col_prediction=PREDICTION)
eval_mae = mae(test_df, scores, col_user=USER, col_item=ITEM, col_rating=RATING, col_prediction=PREDICTION)
eval_exp_var = exp_var(test_df, scores, col_user=USER, col_item=ITEM, col_rating=RATING, col_prediction=PREDICTION)

print("Model:\t" + learn.__class__.__name__,
      "RMSE:\t%f" % eval_rmse,
      "MAE:\t%f" % eval_mae,
      "Explained variance:\t%f" % eval_exp_var,
      "R squared:\t%f" % eval_r2, sep='\n')

Model:	CollabLearner
RMSE:	0.909642
MAE:	0.728852
Explained variance:	0.342464
R squared:	0.336665


In [87]:
# Record results with papermill for tests
sb.glue("map", eval_map)
sb.glue("ndcg", eval_ndcg)
sb.glue("precision", eval_precision)
sb.glue("recall", eval_recall)
sb.glue("rmse", eval_rmse)
sb.glue("mae", eval_mae)
sb.glue("exp_var", eval_exp_var)
sb.glue("rsquared", eval_r2)
sb.glue("train_time", train_time.interval)
sb.glue("test_time", test_time.interval)

In [91]:
train_valid_df.head()

,userID,itemID,rating,timestamp
95682,641,1,5.0,1019443119
380059,2223,1,5.0,974599693
88015,579,1,4.0,975977259
206206,1265,1,4.0,1011716691
920060,5557,1,5.0,959440917


In [92]:
test_df.head()

,userID,itemID,rating,timestamp
681008,4082,1,4.0,965442009
523164,3226,1,4.0,968436164
228818,1387,1,3.0,974765459
71262,479,1,5.0,976218440
607867,3689,1,4.0,966311142


In [90]:
# Make users and movies into separate arrays in the training and test data for Keras input
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

# EmbeddingLayer as a calling class
class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, Embedded):
        Embedded = layers.Embedding(self.n_items, 
                             self.n_factors, 
                             embeddings_initializer='he_normal',
                             embeddings_regularizer=l2(1e-6))(Embedded)
        Embedded = Reshape((self.n_factors,))(Embedded)
        return Embedded


def rec_Model(n_users, n_movies, embedding_size=50):
    #Creating an embedding layer having embedding vectors of user and item ids.
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, embedding_size)(user)
        
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, embedding_size)(movie)
    
    # Dot product for interactions between users and items
    interaction = Dot(axes=1)([u, m])

    # We used Adam for optimizer as learning_rate=0.001
    rec_model = Model(inputs=[user, movie], outputs=interaction)
    opt = Adam(learning_rate=0.001)

    # Computing loss as mean squared error
    rec_model.compile(loss='mean_squared_error', optimizer=opt)

    return rec_model #recommender model

NameError: ignored

In [89]:
from keras.layers import Activation, Concatenate, Dense, Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.regularizers import l2

import matplotlib.pyplot as plt

def NNmodel(n_users, n_movies, embedding_size=50):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, embedding_size)(user)
    
    movie = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, embedding_size)(movie)
    
    # Concat on the embedding layers and dropout layer
    interaction = Concatenate()([u, m])
    interaction = Dropout(0.05)(interaction)

    # Dense layer with relu and sigmoid activation functions for interaction    
    interaction = Dense(10, kernel_initializer='he_normal')(interaction)
    interaction = Activation('relu')(interaction)
    interaction = Dropout(0.5)(interaction)
    
    interaction = Dense(1, kernel_initializer='he_normal')(interaction)
    interaction = Activation('sigmoid')(interaction)

    # Again used Adam with lr=0.001
    NNmodel = Model(inputs=[user, movie], outputs=interaction)
    opt = Adam(learning_rate=0.001)
    NNmodel.compile(loss='mean_squared_error', optimizer=opt)

    return NNmodel